In [1]:
import pickle

In [2]:
model = pickle.load(open('greatest_tree.pkl','rb'))

In [3]:
model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [8]:
unseen_contract = [0, 2, 3,1,100,1]

In [9]:
model.predict([unseen_contract])

array([1])

## EXERCISE

In [10]:
import pandas as pd

In [11]:
clients = pd.read_sas('./data/anagrf_srs_an.sas7bdat', encoding='utf8')

In [12]:
clients.head()

,SEX,TIPO_FINZ_GARN,DAT_NSC,Klient_ID
0,M,1,1980-12-21,K_000001
1,Z,1,1986-12-29,K_000002
2,Z,2,1968-01-27,K_000003
3,Z,1,1982-10-22,K_000004
4,M,1,1984-10-19,K_000005


## Task
- Create a model using `SEX`, `TIPO_FINZ_GARN` and `IMP_RATE_RSD`, `NUM_RATE_SCAD`, `IMP_RATE_SCAD`
to predict whether `NUM_RATE_RSD` is 0 or 1 (as before).
     - First, do group_by contract in the second table.
- **Model:** RandomForestClassifier, (`from sklearn.ensemble import RandomForestClassifier`).
- Do grid search on `n_estimators` and `max_depth`.
